In [1]:
import sys
from dataCopy1 import Data
import numpy as np

if '../src' not in sys.path:
    sys.path.append('../src')

remove_stop = False

this_data = Data()
this_data.load_hieve(file_direct='../datasets/hieve_processed/', remove_stop=True, num_case_limit=46072) 
#46072 is the case limit from the ACL19 paper

#then you will have tuples (session_id, p_id, c_id, p_pos, c_pos, label) in this_data.data_cases
# To get triples in the form (event1_surface, event2_surface, label):
triples = [(this_data.text[int(x[0])][x[3]].lower(), this_data.text[int(x[0])][x[4]].lower(), x[5]) for x in this_data.data_cases]

#Files: 100
46072 rel cases out of 99 articles.
#event_types 3173
label_map [[0. 1.]
 [1. 0.]]   label_weights [9.398612811097511, 0.528094266523005]


In [2]:
count_0 = 0
count_1 = 0
for i in triples:
    if i[2] == 0:
        count_0 += 1
    if i[2] == 1:
        count_1 += 1
print(count_0)    # 43621 triples with false supersub relation
print(count_1)    # 2451 triples with true supersub relation
print(len(triples))
print(type(triples[0]))
print(triples[900])
with open('./triples_46072.tsv', 'w') as f_triples:
    for item in triples:
        f_triples.write(str(item))
        f_triples.write("\n")

43621
2451
46072
<class 'tuple'>
('incidents', 'holding', 1)


In [3]:
my_Ent_list = []
for i in triples:
    my_Ent_list.append(i[0])
    my_Ent_list.append(i[1])
    
Ent_List_set = set(my_Ent_list) #List_set是另外一个列表，里面的内容是List里面的无重复项 
print(len(Ent_List_set))
with open('./HiEve_Ent_List_set.tsv', 'w') as f_Ent:
    for item in Ent_List_set:
        f_Ent.write(item)
        f_Ent.write("\n")

905


In [4]:
import networkx as nx
import node2vec
import datetime

def parse_args():
	'''
	Parses the node2vec arguments.
	'''
	parser = argparse.ArgumentParser(description="Run node2vec.")
#graph/karate.edgelist
	parser.add_argument('--input', nargs='?', default='0Out.tsv',
	                    help='Input graph path')

	parser.add_argument('--output', nargs='?', default='emb/karate.emb',
	                    help='Embeddings path')

	parser.add_argument('--dimensions', type=int, default=128,
	                    help='Number of dimensions. Default is 128.')

	parser.add_argument('--walk-length', type=int, default=80,
	                    help='Length of walk per source. Default is 80.')

	parser.add_argument('--num-walks', type=int, default=10,
	                    help='Number of walks per source. Default is 10.')

	parser.add_argument('--window-size', type=int, default=10,
                    	help='Context size for optimization. Default is 10.')

	parser.add_argument('--iter', default=1, type=int,
                      help='Number of epochs in SGD')

	parser.add_argument('--workers', type=int, default=8,
	                    help='Number of parallel workers. Default is 8.')

	parser.add_argument('--p', type=float, default=1,
	                    help='Return hyperparameter. Default is 1.')

	parser.add_argument('--q', type=float, default=1,
	                    help='Inout hyperparameter. Default is 1.')

	parser.add_argument('--weighted', dest='weighted', action='store_true',
	                    help='Boolean specifying (un)weighted. Default is unweighted.')
	parser.add_argument('--unweighted', dest='unweighted', action='store_false')
	parser.set_defaults(weighted=False)

	parser.add_argument('--directed', dest='directed', action='store_true',
	                    help='Graph is (un)directed. Default is undirected.')
	parser.add_argument('--undirected', dest='undirected', action='store_false')
	parser.set_defaults(directed=False)

	parser.add_argument('--relation', dest='relationed', action='store_true',
						help='Boolean specifying (un)relationed. Default is unrelationed.')
	parser.add_argument('--unrelation', dest='unrelationed', action='store_false')
	parser.set_defaults(relationed=False)		

	return parser.parse_args()

def read_graph():
	'''
	Reads the input network in networkx.
	'''
	'''
	if args.weighted:
		G = nx.read_edgelist(args.input, nodetype=str, data=(('weight',float),), create_using=nx.DiGraph())
	else:
		if args.relationed:
			G = nx.read_edgelist(args.input, nodetype=str, data=(('relation',str),), create_using=nx.DiGraph())
			for edge in G.edges():
				G[edge[0]][edge[1]]['weight'] = 1
		else:
			G = nx.read_edgelist(args.input, nodetype=str, create_using=nx.DiGraph())
			for edge in G.edges():
				G[edge[0]][edge[1]]['weight'] = 1
	
	if not args.directed:
		G = G.to_undirected()
	'''
	count = 0
	G = nx.read_edgelist('0Out.tsv', nodetype=str, data=(('relation',str),), create_using=nx.DiGraph())
	for edge in G.edges():
		G[edge[0]][edge[1]]['weight'] = 1
		count = count+1
		#print(count)
	return G


'''
Pipeline for representational learning for all nodes in a graph.
'''

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
G = read_graph()
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2019-09-23 18:09:09
2019-09-23 18:09:37


In [5]:
import itertools
def subgraph_gen(cutoff):
    neighbor = []
    count_item = 0
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    for item in Ent_List_set:
        try:
            item_neighbors = nx.single_source_shortest_path_length(G, item, cutoff=cutoff)
        except:
            print("Error: Source " + item + " is not in G")
        else:
            neighbor.append(list(item_neighbors.keys()))
            count_item += 1

        if count_item % 50 == 0:
            print(count_item)
            print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    merged = list(itertools.chain.from_iterable(neighbor))
    print(len(merged))
    merged_set = set(merged) #List_set是另外一个列表，里面的内容是
    with open('./HiEve_merged_' + str(cutoff) + '_set.tsv', 'w') as f_out:
        for item in merged_set:
            f_out.write(item)
            f_out.write("\n")

    print(len(merged_set))
    SubG = G.subgraph(list(merged_set))
    nx.write_edgelist(SubG, 'SubG_' + str(cutoff) + '.tsv')
    
subgraph_gen(10)

2019-09-23 18:09:51
50
2019-09-23 18:11:17
100
2019-09-23 18:12:44
150
2019-09-23 18:14:13
Error: Source shieldssearched is not in G
200
2019-09-23 18:15:41
250
2019-09-23 18:17:06
300
2019-09-23 18:18:39
350
2019-09-23 18:20:03
400
2019-09-23 18:21:30
450
2019-09-23 18:22:57
500
2019-09-23 18:24:28
550
2019-09-23 18:25:57
600
2019-09-23 18:27:25
650
2019-09-23 18:28:54
700
2019-09-23 18:30:25
750
2019-09-23 18:31:51
800
2019-09-23 18:33:15
850
2019-09-23 18:34:49
900
2019-09-23 18:36:19
268082477
297373
